In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class JH_Forces():
    
    def __init__(self, li=[None]):
        '''Constructor for Forces Calculation as per J.Heikkala Method
           Pass: List of Feed per Tooth, Cutting Speed, Diameter of Cutter, Depth of Cut, Width of Cut, 
                 Number of Teeth, Approach Angle, Radial Rake Angle, Axial Rake Angle and y.'''
        # Parameters
        self.li           = li
        self.feed         = self.li[0]       # Feed in mm/tooth
        self.cut_speed    = self.li[1]       # Cut Speeed in m/min
        self.dia_cutter   = self.li[2]       # Diameter of Cutter in mm 
        self.doc          = self.li[3]       # Depth of Cut in mm
        self.woc          = self.li[4]       # Width of Cut in mm
        self.z            = self.li[5]       # Number of teeth
        self.approach_ang = self.li[6]       # Approach angle in degrees
        self.gamma_r      = self.li[7]       # Radial Rake Angle
        self.gamma_a      = self.li[8]       # Axial Rake Angle
        self.y            = self.li[9]       # Y distance in mm
        
        # Dataframes
        self.df = pd.read_csv('Specific_Cutting_Forces.csv')   # Read CSV File
        
    def get_forces(self):
        '''Calculates Forces, Power and Torques.
           Pass: None
           Return: DataFrame of Forces, Torques and Power.'''
        
        li = ['Material','Tangential Force (N)', 'Radial Force (N)', 'Axial Force (N)', 'Torque (N-m)', 'Power (kW)']
        df = pd.DataFrame(columns = li)
        
        # Calculate Speed
        speed = (self.cut_speed * 1000)/(np.pi * self.dia_cutter)
        
        # Calculate Rotating Angles
        psi_min = np.pi*0.5 - np.arcsin((2*self.y)/self.dia_cutter)
        psi_max = np.pi*0.5 + np.arcsin(2*(self.woc - self.y)/self.dia_cutter)
        alpha = psi_max-psi_min
        
        # Number of Teeths in COntact
        k = (alpha*self.z)/(2*np.pi) +1
        k = int(k)
        
        # Calculate Forces 
        li_tang = []
        li_rad = []
        li_axi = []
        li_torq = []
        li_power =[]
        for ksi, q in zip(self.df['Specific cutting force'], self.df['r']):
            r = 1-q
            ksi = ksi/1000
            
            # Calculate Coefficients
            A = self.doc * ksi * (self.feed**r) * (np.sin(self.approach_ang * (np.pi/180)))**(r-1) * (1 - (0.01*self.gamma_r))
            B = 0.44 - (0.013*self.gamma_r)
            C = ((self.feed + (self.doc/np.tan(self.approach_ang*(np.pi/180))))/self.doc) * (0.88 - (0.013*self.gamma_a))
            
            # Calculate Angles
            li_ang = []
            for i in range(int(k)):
                psi = psi_min + ((2*np.pi)/self.z)*(i+1-1)
                li_ang.append(psi)
            
            # Calculate Forces
            li_forces =[]
            for i in range(int(k)):
                force = A * (np.sin(li_ang[i]))**r
                li_forces.append(force)
            
            # Tangential, Radial and Axial Forces
            tang_force = sum(li_forces) * 1000
            radial_force = tang_force * B
            axial_force = tang_force * C
            
            # Torque
            torque = tang_force * (self.dia_cutter/2000)
            
            # Power 
            power = (2*np.pi*speed*torque)/60000
            
            # Append to List
            li_tang.append(tang_force)
            li_rad.append(radial_force)
            li_axi.append(axial_force)
            li_torq.append(torque)
            li_power.append(power)
        
        # Update the Dataframe
        df['Material'] = self.df['Material']
        df['Tangential Force (N)'] = li_tang
        df['Radial Force (N)'] = li_rad
        df['Axial Force (N)'] = li_axi
        df['Torque (N-m)'] = li_torq
        df['Power (kW)'] = li_power
        
        return df # Return Dataframe

In [3]:
feed = 0.15           # Feed in mm/tooth
cut_speed = 144        # Cut Speeed in m/min
dia_cutter = 100         # Diameter of Cutter in mm 
doc = 2               # Depth of Cut in mm
woc =80                # Width of Cut in mm
z = 8                   # Number of teeth
approach_ang = 67       # Approach angle in degrees
gamma_r = -5            # Radial Rake Angle
gamma_a = -7             # Axial Rake Angle
ksi = 1.79            # Specific Cutting force in kN/mm2
y = woc/2               # y value - woc/2
r = 0.8                # r factor given tables

li = [feed, cut_speed, dia_cutter, doc, woc, z, approach_ang, gamma_r, gamma_a, y]

In [4]:
force = JH_Forces(li)

In [5]:
df = force.get_forces()

In [6]:
df.head()

,Material,Tangential Force (N),Radial Force (N),Axial Force (N),Torque (N-m),Power (kW)
0,Steel Unalloyed C = 0.1 - 0.25%,1953.381269,986.457541,947.370473,97.669063,4.688115
1,Steel Unalloyed C = 0.25 - 0.55%,2083.606687,1052.221377,1010.528504,104.180334,5.000656
2,Steel Unalloyed C = 0.55 - 0.80%,2213.832105,1117.985213,1073.686536,110.691605,5.313197
3,Steel-Low alloyed (alloying elements <= 5%) No...,2213.832105,1117.985213,1073.686536,110.691605,5.313197
4,Steel-Low alloyed (alloying elements <=5%) Har...,2474.282941,1249.512885,1200.002599,123.714147,5.938279


In [10]:
df.to_excel('Result Dataframe.xlsx')

# -----------------------------------------------------------------------------------------------------------------

In [9]:
class JH_Forces():
    
    def __init__(self, li=[None]):
        '''Constructor for Forces Calculation as per J.Heikkala Method
           Pass: List of Feed per Tooth, Cutting Speed, Diameter of Cutter, Depth of Cut, Width of Cut, 
                 Number of Teeth, Approach Angle, Radial Rake Angle, Axial Rake Angle and y.'''
        # Parameters
        self.li           = li
        self.feed         = self.li[0]       # Feed in mm/tooth
        self.dia_cutter   = self.li[1]       # Diameter of Cutter in mm 
        self.doc          = self.li[2]       # Depth of Cut in mm
        self.woc          = self.li[3]       # Width of Cut in mm
        self.z            = self.li[4]       # Number of teeth
        self.approach_ang = self.li[5]       # Approach angle in degrees
        self.gamma_r      = self.li[6]       # Radial Rake Angle
        self.gamma_a      = self.li[7]       # Axial Rake Angle
        self.y            = self.li[8]       # Y distance in mm
        
        # Dataframes
        self.df = pd.read_csv('Specific_Cutting_Forces.csv')   # Read CSV File
        
    def get_forces(self):
        '''Calculates Forces, Power and Torques.
           Pass: None
           Return: DataFrame of Forces, Torques and Power.'''
        
        li = ['Material','Specific cutting force','Cutting Speed (m/min)','Speed (RPM)','Tangential Force (N)', 'Radial Force (N)', 'Axial Force (N)', 'Torque (N-m)', 'Power (kW)']
        df = pd.DataFrame(columns = li)
        
        # Lists to Store
        li_tang = []
        li_rad = []
        li_axi = []
        li_torq = []
        li_power =[]
        li_speed = []
        li_cut_speed = []
        
        for ksi, q, speed in zip(self.df['Specific cutting force'], self.df['r'], self.df['Cutting Speed']):
            r = 1-q
            ksi = ksi/1000
            
            # Calculate Speed
            rot_speed = (speed * 1000)/(np.pi * self.dia_cutter)

            # Calculate Rotating Angles
            psi_min = np.pi*0.5 - np.arcsin((2*self.y)/self.dia_cutter)
            psi_max = np.pi*0.5 + np.arcsin(2*(self.woc - self.y)/self.dia_cutter)
            alpha = psi_max-psi_min

            # Number of Teeths in COntact
            k = (alpha*self.z)/(2*np.pi) +1
            k = int(k)

            # Calculate Forces 
        
            # Calculate Coefficients
            A = self.doc * ksi * (self.feed**r) * (np.sin(self.approach_ang * (np.pi/180)))**(r-1) * (1 - (0.01*self.gamma_r))
            B = 0.44 - (0.013*self.gamma_r)
            C = ((self.feed + (self.doc/np.tan(self.approach_ang*(np.pi/180))))/self.doc) * (0.88 - (0.013*self.gamma_a))
            
            # Calculate Angles
            li_ang = []
            for i in range(int(k)):
                psi = psi_min + ((2*np.pi)/self.z)*(i+1-1)
                li_ang.append(psi)
            
            # Calculate Forces
            li_forces =[]
            for i in range(int(k)):
                force = A * (np.sin(li_ang[i]))**r
                li_forces.append(force)
            
            # Tangential, Radial and Axial Forces
            tang_force = sum(li_forces) * 1000
            radial_force = tang_force * B
            axial_force = tang_force * C
            
            # Torque
            torque = tang_force * (self.dia_cutter/2000)
            
            # Power 
            power = (2 * np.pi * rot_speed * torque)/60000
        
            # Append to List
            li_tang.append(np.ceil(tang_force))
            li_rad.append(np.ceil(radial_force))
            li_axi.append(np.ceil(axial_force))
            li_torq.append(np.ceil(torque))
            li_power.append(np.ceil(power))
            li_speed.append(np.ceil(rot_speed))
            li_cut_speed.append(np.ceil(speed))
            
        
        # Update the Dataframe
        df['Material'] = self.df['Material']
        df['Specific cutting force'] = self.df['Specific cutting force']
        df['Tangential Force (N)'] = li_tang
        df['Radial Force (N)'] = li_rad
        df['Axial Force (N)'] = li_axi
        df['Torque (N-m)'] = li_torq
        df['Power (kW)'] = li_power
        df['Cutting Speed (m/min)'] = li_cut_speed
        df['Speed (RPM)'] = li_speed
        
        return df # Return Dataframe

In [24]:
feed = 0.15           # Feed in mm/tooth
dia_cutter = 100        # Diameter of Cutter in mm 
doc = 1               # Depth of Cut in mm
woc =100                # Width of Cut in mm
z = 6                   # Number of teeth
approach_ang = 45       # Approach angle in degrees
gamma_r = -5            # Radial Rake Angle
gamma_a = -7             # Axial Rake Angle
y = woc/2               # y value - woc/2

li = [feed, dia_cutter, doc, woc, z, approach_ang, gamma_r, gamma_a, y]

In [25]:
force = JH_Forces(li)

In [26]:
df = force.get_forces()

In [27]:
df.head()

,Material,Specific cutting force,Cutting Speed (m/min),Speed (RPM),Tangential Force (N),Radial Force (N),Axial Force (N),Torque (N-m),Power (kW)
0,Steel Unalloyed C = 0.1 - 0.25%,1500,490.0,1560.0,744.0,376.0,830.0,38.0,7.0
1,Steel Unalloyed C = 0.25 - 0.55%,1600,440.0,1401.0,793.0,401.0,886.0,40.0,6.0
2,Steel Unalloyed C = 0.55 - 0.80%,1700,415.0,1321.0,843.0,426.0,941.0,43.0,6.0
3,Steel-Low alloyed (alloying elements <= 5%) No...,1700,345.0,1099.0,843.0,426.0,941.0,43.0,5.0
4,Steel-Low alloyed (alloying elements <=5%) Har...,1900,225.0,717.0,942.0,476.0,1052.0,48.0,4.0


In [28]:
df.to_excel('80 100/Result_Dataframe_100.xlsx')

In [2]:
df1 = pd.read_csv('Specific_Cutting_Forces.csv')  

In [5]:
df1.head()

,Material,Class Iso,Specific cutting force,Brinnel Hardness,r,Cutting Speed
0,Steel Unalloyed C = 0.1 - 0.25%,P,1500,125,0.25,490
1,Steel Unalloyed C = 0.25 - 0.55%,P,1600,150,0.25,440
2,Steel Unalloyed C = 0.55 - 0.80%,P,1700,170,0.25,415
3,Steel-Low alloyed (alloying elements <= 5%) No...,P,1700,175,0.25,345
4,Steel-Low alloyed (alloying elements <=5%) Har...,P,1900,300,0.25,225


In [8]:
np.ceil(17.17)

18.0